In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from darts import TimeSeries
from darts.models import NBEATSModel
from darts import datasets

from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
import nsepy
from datetime import date

c:\Users\Mukund Mishra\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nifty100list = pd.read_csv('ind_nifty100list.csv')['Symbol']
nifty100list = np.array(nifty100list)

In [69]:
dictClose = {}
for i in nifty100list:
    df = pd.DataFrame(nsepy.get_history(symbol = nifty100list[0], start = date(2020, 10, 1), end = date.today() ))['Close']
    df = np.array(df)
    df = TimeSeries.from_values(df, columns=None, fillna_value=None, static_covariates=None, hierarchy=None)
    dictClose[i] = df

In [75]:
totallist = []
scalerdict = {}
for i in dictClose:
    datascaler = Scaler()
    dictClose[i] = datascaler.fit_transform(dictClose[i])
    scalerdict[i] = datascaler
    totallist.append(dictClose[i])

In [76]:
finalmodel = NBEATSModel(input_chunk_length = 150, output_chunk_length = 30, n_epochs = 200, random_state = 0)

In [77]:
finalmodel.fit(totallist, verbose=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 6.1 M 
---------------------------------------------------
6.1 M     Trainable params
1.3 K     Non-trainable params
6.1 M     Total params
48.686    Total estimated model params size (MB)


Epoch 0:  67%|██████▋   | 117/175 [00:18<00:08,  6.48it/s, loss=0.00265, train_loss=0.00264]

c:\Users\Mukund Mishra\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# can we train model by different name and predict by different name ???
#"YES"
# from now save model by model.save in darts in pt format 
# and load model by 
finalmodel.save('final_model.pt')
